In [1]:
# source: https://www.codesansar.com/numerical-methods/
#         runge-kutta-fourth-order-rk4-python-program.htm

import os
import cmath #To help us out with the complex square root
import numpy as np #For the arrays
import matplotlib.pyplot as plt #Visualization

# deleting profile and radmass files
# os.remove("radmass.txt")
# os.remove("profile.txt")

# the TOV GR equation

def b1(r,P,f,b,params):
    PCC=params[0]
    Q=params[1]
    eta=params[2]
    Y=params[3]
    return (1-f)*b/(r*f)

def P1(r,P,f,b,params):
    PCC=params[0]
    Q=params[1]
    eta=params[2]
    Y=params[3]
    return -(eden(P,params)+P) * b1(r,P,f,b,params)/(2*b) - 2*sig(P,params,b,f,r)/r

def f1(r,P,f,b,params):
    PCC=params[0]
    Q=params[1]
    eta=params[2]
    Y=params[3]
    A = r*b/f*(P*r**2+4*kappa)-3*eta*Q**2*r
    B = 3*(1-f)*eta*Q**2 
    B = B + b/f*(6*r**2*f*P + (1+f)*r**2*eden(P,params) -4*kappa*(1-f) -4*r**2*f*sig(P,params,b,f,r))
    return -B/A

def om1(r,P,f,b,om,ka,params):
    PCC=params[0]
    Q=params[1]
    eta=params[2]
    Y=params[3]
    return 6*ka/r**4

def ka1(r,P,f,b,om,ka,params):
    PCC=params[0]
    Q=params[1]
    eta=params[2]
    Y=params[3]
    # om2 = K_1 om1 + K_2 om
    K_2 = 4*b*(eden(P,params)+P-sig(P,params,b,f,r))
    K_2 = K_2/(f*(-Q**2*eta*f+b*(4*kappa+r**2*P)))
    K_1 = 4*Q**2*eta*f**2
    K_1 = K_1 + b*( -4*f*(4*kappa+r**2*P) + r**2*(eden(P,params)+P) )
    K_1 = K_1/(r*f*(-Q**2*eta*f+b*(4*kappa+r**2*P)))
    return (4/r + K_1)*ka + (r**4/6)*K_2*om

In [2]:
# this is only for a single PCC
def single_PCC(PCC,Qinf,eta,Y,bCC,radmassdata,profile):
    # define initial parameters
    rCC = .000000001 # radius near center in m--the starting point
    rmax = 100000. # radius at far distances in m
    # PCC = PCC_Input # pressure at the center in MeV / fm^3
    fCC = 1. # metric function f(r) at the center
    # bCC = bCC_Input # metric function b(r) at the center

    h = 1. # h-step

    UBQ = np.sqrt(4*kappa/3)
    Qinf = Qinf*UBQ #.29*UBQ   # not more than .29*UBQ is save up to PCC=1200
    # eta = eta_Input
    Q = Qinf
    params = np.array([PCC,Q,eta,Y])
    # print(params[0]/UBQ,eta)

    # calculate the surface values
    output=RungeKutta(rCC,bCC,PCC,fCC,params,h,0)
    # print(output)

    # at the surface, b = 1-2Gm/r, which is different to the result
    rSurface=output[0]
    bSurface=output[1]
    mSurface=output[3]
    # print(PCC, rSurface, mSurface, 
        #   GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ) 
    bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface

    # NOTICE: cannot use it since R and M chaages
    # instead we follow the paper by Cisterna PRD92,044050(2015)
    # i.e. bCC will not be modified, but Q is modified instead 

    # Or rather inputting Qinf fixed instead and calculate Q,
    # which is done by only modifying bCC,
    # then Q=Q/np.sqrt(bCorrection)

    # So, we redefine bCC as follows
    bCorrection=bSurfaceTarget/bSurface  # bCorrection=1/binf
    # print(abs(bCorrection-1))
    # bCC and Q will be modified into bCC*bCorrection
    # and Q*np.sqrt(bCorrection) 

    # if abs(bCorrection) not near 1, then recalculate
    # bCC=bCC*bCorrection
    # Q=Q/np.sqrt(bCorrection)
    while (abs(bCorrection-1)>10**(-3)):
        bCC=bCC*bCorrection
        Q=Q/np.sqrt(bCorrection)
        params = np.array([PCC,Q,eta,Y])
        # print(params[0]/UBQ,eta)
        output=RungeKutta(rCC,bCC,PCC,fCC,params,h,0)
        # print(output)
        rSurface=output[0]
        bSurface=output[1]
        mSurface=output[3]
        # print(PCC, rSurface, mSurface, 
        #         GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ) 
        bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
        bCorrection=bSurfaceTarget/bSurface
        # print(abs(bCorrection-1))
        # the result R and M does change
        # so this loop is essential
    # We NEED to redefine both b and Q
    # so that R and M don'o't change
    # --they change if we didn't
    # redefine Q

    rSurface=output[0]
    bSurface=output[1]
    mSurface=output[3]
    print(PCC, rSurface, mSurface, 
                GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ) 
    
    if profile==1:
        output=RungeKutta(rCC,bCC,PCC,fCC,params,h,1)
    if radmassdata == 1:
        print(PCC, (eden(PCC,params)/1000), (rSurface/1000), mSurface, 
            GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ,
            file=open('radmassST.dat', 'a'))

    output = np.array([PCC, (eden(PCC,params)/1000), (rSurface/1000), mSurface, GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ])
    return output

# define the Runge-Kutta 4th order for the problem
# if we want to print the profile, set profile=1
# if we not, set profile=0
def RungeKutta(rCC,bCC,PCC,fCC,params,h,profile):
    # input initial values
    r0 = rCC
    b0 = bCC
    P0 = PCC
    f0 = fCC
    m0 = (1-f0)*r0/(2*GS*MSS)
    while (P0 > 0.):
        # print(r0, b0, P0, m0)
        if profile == 1:
            print(r0, b0, P0, m0, file=open('profileST.dat', 'a'))
        # calculate k1
        r01 = r0
        b01 = b0
        P01 = P0
        f01 = f0
        k1_b = h * b1(r01,P01,f01,b01,params)
        k1_P = h * P1(r01,P01,f01,b01,params)
        k1_f = h * f1(r01,P01,f01,b01,params)
        # calculate k2
        r01 = r0 + h/2
        b01 = b0 + k1_b/2
        P01 = P0 + k1_P/2
        f01 = f0 + k1_f/2
        k2_b = h * b1(r01,P01,f01,b01,params)
        k2_P = h * P1(r01,P01,f01,b01,params)
        k2_f = h * f1(r01,P01,f01,b01,params)
        # calculate k3
        r01 = r0 + h/2
        b01 = b0 + k2_b/2
        P01 = P0 + k2_P/2
        f01 = f0 + k2_f/2
        k3_b = h * b1(r01,P01,f01,b01,params)
        k3_P = h * P1(r01,P01,f01,b01,params)
        k3_f = h * f1(r01,P01,f01,b01,params)
        # calculate k4
        r01 = r0 + h
        b01 = b0 + k3_b
        P01 = P0 + k3_P
        f01 = f0 + k3_f
        k4_b = h * b1(r01,P01,f01,b01,params)
        k4_P = h * P1(r01,P01,f01,b01,params)
        k4_f = h * f1(r01,P01,f01,b01,params)
        # calculate the next r0, P0, m0, and b0
        r0 = r0 + h
        b0 = b0 + (k1_b+2*k2_b+2*k3_b+k4_b)/6
        P0 = P0 + (k1_P+2*k2_P+2*k3_P+k4_P)/6
        f0 = f0 + (k1_f+2*k2_f+2*k3_f+k4_f)/6
        m0 = (1-f0)*r0/(2*GS*MSS)
        if P0 > 2*PCC: 
            print(PCC,"dP/dr>0")
            break
    # the results at the surface
    if profile == 1:
        print(r0, b0, P0, m0, file=open('profileST.dat', 'a'))
    return np.array([r0,b0,P0,m0,f0])

In [3]:
def single_momin(PCC,Qinf,eta,Y,bCC,radmassdata,profile):
    # define initial parameters
    rCC = .000000001 # radius near center in m--the starting point
    rmax = 100000. # radius at far distances in m
    fCC = 1. # metric function f(r) at the center

    h = 1. # h-step

    UBQ = np.sqrt(4*kappa/3)
    Qinf = Qinf*UBQ 
    Q = Qinf
    params = np.array([PCC,Q,eta,Y])
    output=RungeKutta(rCC,bCC,PCC,fCC,params,h,0)

    rSurface=output[0]
    bSurface=output[1]
    mSurface=output[3]
    bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
    bCorrection=bSurfaceTarget/bSurface

    Pcont=PCC
    while (abs(bCorrection-1)>10**(-3)):
        # print("tes1",abs(bCorrection-1))
        bCorr_old=bCorrection
        bCC=bCC*bCorrection
        Q=Q/np.sqrt(bCorrection)
        params = np.array([PCC,Q,eta,Y])
        output=RungeKutta(rCC,bCC,PCC,fCC,params,h,0)
        PSurface=output[2]
        if PSurface>PCC:
            # print("tes1: dP/dr>0") 
            Pcont=PSurface
            break
        rSurface=output[0]
        bSurface=output[1]
        mSurface=output[3]
        bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
        bCorrection=bSurfaceTarget/bSurface
        bCorr_new=bCorrection
        if abs(bCorr_new-1)>abs(bCorr_old-1):
            print(abs(bCorr_new-1),abs(bCorr_old-1))
            break
    
           
    if Pcont>PCC:
        print(PCC, PSurface)
        output = np.array([PCC, PSurface])
    else:
        rSurface=output[0]
        bSurface=output[1]
        mSurface=output[3]
        # print(PCC, rSurface, mSurface, 
        #             GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ) 

        # kelar hitung TOV, lanjut ke momen inersia
        omCC = 1.e-8
        omCC = omCC + (8./5)*GS*PI*rCC**2*(PCC+eden(PCC,params))*omCC
        kaCC = (8./15)*GS*PI*rCC**2*(PCC+eden(PCC,params))*omCC
        output = RungeKutta1(rCC,bCC,PCC,fCC,omCC,kaCC,params,h,0)
        # at the surface, moment of inertia is momin
        rSurface=output[0]
        bSurface=output[1]
        mSurface=output[3]
        # bSurfaceTarget = 1-2*GS*MSS*mSurface/rSurface
        # bCorrection=bSurfaceTarget/bSurface
        omSurface=output[5]
        kaSurface=output[6]
        zeta=1/(omSurface+2*kaSurface/(rSurface**3))
        momin=kaSurface*zeta/GS
        momin1=momin/(MSS*mSurface*rSurface**2)  # I/MR^2 tanpa satuan, diplot thd compactness
        momin2=momin1*1.98892e33*1.e10*mSurface*(rSurface*1.e-3)**2/1.e45 # I satuan 10^45 g cm^2 diplot thd massa
        print(PCC, rSurface, mSurface, GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ, momin1, momin2, zeta)
        if Q/UBQ >= 1: print("Q>UBQ")
        
        if profile==1:
            output=RungeKutta1(rCC,bCC,PCC,fCC,omCC*zeta,kaCC*zeta,params,h,1)
        if radmassdata == 1 and Q/UBQ < 1:
            print(PCC, (eden(PCC,params)/1000), (rSurface/1000), mSurface, 
                GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ,  
                momin1, momin2, omCC*zeta, kaCC*zeta,
                file=open('radmassSTmomin.dat', 'a'))

        output = np.array([PCC, (eden(PCC,params)/1000), (rSurface/1000), mSurface, GS*MSS*mSurface/rSurface, bCC, Qinf/UBQ, Q/UBQ,  momin1, momin2, omCC*zeta, kaCC*zeta])
    return output

def RungeKutta1(rCC,bCC,PCC,fCC,omCC,kaCC,params,h,profile):
    # input initial values
    r0 = rCC
    b0 = bCC
    P0 = PCC
    f0 = fCC
    m0 = (1-f0)*r0/(2*GS*MSS)
    om0 = omCC
    ka0 = kaCC
    while (P0 > 0.):
        if profile == 1:
            print(r0, b0, P0, m0, om0, ka0, file=open('profileSTmomin.dat', 'a'))
        # calculate k1
        r01 = r0
        b01 = b0
        P01 = P0
        f01 = f0
        om01 = om0
        ka01 = ka0
        k1_b = h * b1(r01,P01,f01,b01,params)
        k1_P = h * P1(r01,P01,f01,b01,params)
        k1_f = h * f1(r01,P01,f01,b01,params)
        k1_om = h * om1(r01,P01,f01,b01,om01,ka01,params)
        k1_ka = h * ka1(r01,P01,f01,b01,om01,ka01,params)
        # calculate k2
        r01 = r0 + h/2
        b01 = b0 + k1_b/2
        P01 = P0 + k1_P/2
        f01 = f0 + k1_f/2
        om01 = om0 + k1_om/2
        ka01 = ka0 + k1_ka/2
        k2_b = h * b1(r01,P01,f01,b01,params)
        k2_P = h * P1(r01,P01,f01,b01,params)
        k2_f = h * f1(r01,P01,f01,b01,params)
        k2_om = h * om1(r01,P01,f01,b01,om01,ka01,params)
        k2_ka = h * ka1(r01,P01,f01,b01,om01,ka01,params)
        # calculate k3
        r01 = r0 + h/2
        b01 = b0 + k2_b/2
        P01 = P0 + k2_P/2
        f01 = f0 + k2_f/2
        om01 = om0 + k2_om/2
        ka01 = ka0 + k2_ka/2
        k3_b = h * b1(r01,P01,f01,b01,params)
        k3_P = h * P1(r01,P01,f01,b01,params)
        k3_f = h * f1(r01,P01,f01,b01,params)
        k3_om = h * om1(r01,P01,f01,b01,om01,ka01,params)
        k3_ka = h * ka1(r01,P01,f01,b01,om01,ka01,params)
        # calculate k4
        r01 = r0 + h
        b01 = b0 + k3_b
        P01 = P0 + k3_P
        f01 = f0 + k3_f
        om01 = om0 + k3_om
        ka01 = ka0 + k3_ka
        k4_b = h * b1(r01,P01,f01,b01,params)
        k4_P = h * P1(r01,P01,f01,b01,params)
        k4_f = h * f1(r01,P01,f01,b01,params)
        k4_om = h * om1(r01,P01,f01,b01,om01,ka01,params)
        k4_ka = h * ka1(r01,P01,f01,b01,om01,ka01,params)
        # calculate the next r0, P0, m0, and b0
        r0 = r0 + h
        b0 = b0 + (k1_b+2*k2_b+2*k3_b+k4_b)/6
        P0 = P0 + (k1_P+2*k2_P+2*k3_P+k4_P)/6
        f0 = f0 + (k1_f+2*k2_f+2*k3_f+k4_f)/6
        om0 = om0 + (k1_om+2*k2_om+2*k3_om+k4_om)/6
        ka0 = ka0 + (k1_ka+2*k2_ka+2*k3_ka+k4_ka)/6
        m0 = (1-f0)*r0/(2*GS*MSS)
    # the results at the surface
    if profile == 1:
        print(r0, b0, P0, m0, om0, ka0, file=open('profileSTmomin.dat', 'a'))
    return np.array([r0,b0,P0,m0,f0,om0,ka0])

In [4]:
# some constants
GS = 1.325 * 10**(-12) # Newton constant in m^4 / MeV fm^3
MSS = 1.1155 * 10**(15) # Sun's mass in MeV m^3 / fm^3
PI = np.pi 
HC = 197.327 # hc=1=197.327 MeV fm
kappa = 1/(16*PI*GS)

UBQ = np.sqrt(4*kappa/3)

# PCC = 300. # pressure at the center in MeV / fm^3
# bCC = 1. # metric function b(r) at the center
# LBQ2 = 12*PCC*kappa*bCC/(abs(eta)*(3*PCC-eden(PCC)))
# UBQ2 = 4*kappa*bCC/(3*abs(eta))
# print(Qinf**2/LBQ2,Qinf**2/UBQ2)


# define energy density as function of pressure

def dedP(P,params):
    dP = 0.00001
    x1 = P-2*dP
    x2 = P-dP
    x3 = P+dP
    x4 = P+2*dP
    dedP = eden(x1,params) - 8*eden(x2,params) + 8*eden(x3,params) - eden(x4,params)
    dedP = dedP/(12*dP)
    return dedP

# choose eos
def eden(P0,params):
    lmdbar = 10. # dimensionless; (0,0.1,5,10,20,50,100)
    Bag = 145.**4/(HC**3) # satuan MeV/fm^3
    lambda2 = 4*Bag*lmdbar
    return 3*P0 + 4*Bag + (4*lambda2/(np.pi)**2)*(1-np.sqrt(1+(np.pi**2/lambda2)*(P0 + Bag)))
def sig(P0,params,b,f,r):
    Y=params[3]
    m=(1-f)*r/(2*GS*MSS)
    return Y*GS*MSS*m/r

In [5]:
# single_momin(PCC,Qinf,eta,Y,b0,radmass,profil)
# Q and Qinf in the code is rescaled as Q -> Q*UBQ   # not more than .29 is save up to PCC=1200

output=single_momin(625,.04,-1,0.,1.,0,0)
output=single_momin(625,.04,-1,-1.,1.,0,0)

625 16655.000000001 4.1992129330229675 0.37265651068703454 0.0302609886558849 0.04 0.22994206908919937 0.6274565947284242 14.536443831098936 0.0017045839227741472
625 16697.000000001 4.221537210370674 0.37369529284140085 0.029901015134583108 0.04 0.23132204601365897 0.6301097240991574 14.749626155074528 0.0016742603331368533


In [6]:
# # if we want to print the profile, run this  
# if os.path.exists('profileST.dat'):
#     os.remove('profileST.dat')

# if os.path.exists('profileSTmomin.dat'):
#     os.remove('profileSTmomin.dat')

# if os.path.exists('radmassST.dat'):
#     os.remove('radmassST.dat')

# if os.path.exists('radmassSTmomin.dat'):
#     os.remove('radmassSTmomin.dat')


# Qinf = .0 # in the code Q=Q*UBQ   # not more than .29 is save up to PCC=1200
# eta = -1
# Y = 0.

# output=single_PCC(55,Qinf,eta,Y,1.,1,1)
# output=single_momin(55,Qinf,eta,Y,1.,1,1)

# Qinf = None
# eta = None
# Y = None

In [7]:
if os.path.exists('radmassSTmomin.dat'):
    os.remove('radmassSTmomin.dat')
    

Qinf = .0
eta = -1
Y = 0.
for x in range(1, 5, 1):
    single_momin(x,Qinf,eta,Y,1.,1,0)
for x in range(5, 801, 5):
    single_momin(x,Qinf,eta,Y,1.,1,0)
Qinf = None
eta = None
Y = None

1 4837.000000001 0.0517991359367 0.015828212814146037 0.9526602074223 0.0 0.0 0.4050773414131416 0.009764049129857271 0.3528206653043836
2 6696.000000001 0.13798221482455417 0.030457420526242007 0.9091724747219446 0.0 0.0 0.40993173292594837 0.050440903272420436 0.3360332592575212
3 8032.000000001 0.23914672989781607 0.044007449550709565 0.8691316929533837 0.0 0.0 0.4145711226473245 0.1272120375701418 0.32052231298487677
4 9089.000000001 0.3480038810775584 0.05659179077765596 0.8321596579527455 0.0 0.0 0.4190104346417099 0.23958435090139998 0.30615456319759093
5 9964.000000001 0.46047035181378226 0.06830514327768916 0.7979406366468728 0.0 0.0 0.4232601706660856 0.38485207102625363 0.2928169254361523
10 12871.000000001 1.014362217835327 0.11648398698964069 0.6593310406128691 0.0 0.0 0.4420804708932215 1.4775299410222633 0.23842366623190492
15 14549.000000001 1.4973497661298736 0.15211623513341332 0.5592933565133432 0.0 0.0 0.45757170183189305 2.8844699558825124 0.19881776138549964
20 15